# UVPFL - Complete Implementation

This notebook runs the entire UVPFL pipeline step by step.


In [ ]:
import sys
import os
from pathlib import Path

# Add project to path
project_path = Path('/home/arunabha/Shini/UVPFL')
sys.path.insert(0, str(project_path))
os.chdir(project_path)

import numpy as np
import tensorflow as tf

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ GPU available: {gpus[0].name}")
else:
    print("⚠️  No GPU available, using CPU")

print(f"TensorFlow version: {tf.__version__}")


In [ ]:
# Run environment verification
!python3 verify_environment.py


In [ ]:
from data_preprocessing import UVPFLDataPreprocessor

# Check if already processed
if Path('processed_data/config.json').exists():
    print("✅ Processed data already exists!")
else:
    print("Running data preprocessing...")
    preprocessor = UVPFLDataPreprocessor(
        dataset_path='360dataset',
        output_path='processed_data'
    )
    preprocessor.process_all_data()
    preprocessor.create_train_test_split(test_ratio=0.2)
    print("✅ Data preprocessing complete!")


In [ ]:
from data_loader import UVPFLDataLoaderFast

loader = UVPFLDataLoaderFast(
    dataset_path='360dataset',
    processed_path='processed_data',
    batch_size=4
)

train_gen = loader.train_generator()
frames, targets = next(train_gen)

print(f"✅ Data loader works!")
print(f"   Frames: {frames.shape}, Targets: {targets.shape}")
print(f"   Train samples: {len(loader.train_samples)}")
print(f"   Test samples: {len(loader.test_samples)}")


In [ ]:
from model import UVPFLModel

model = UVPFLModel(
    model_type='resnet50',
    freeze_backbone=True,
    learning_rate=0.001
)

model.summary()


In [ ]:
# Test forward pass
dummy_input = np.random.rand(2, 30, 120, 240, 3).astype(np.float32)
output = model.predict(dummy_input)
print(f"Input: {dummy_input.shape}, Output: {output.shape}")


In [ ]:
# Train model
from model import get_callbacks

train_gen = loader.train_generator(shuffle=True)
val_gen = loader.test_generator()

callbacks = get_callbacks(
    model_name='uvpfl_resnet50',
    checkpoint_dir='checkpoints',
    log_dir='logs'
)

history = model.model.fit(
    train_gen,
    steps_per_epoch=100,
    epochs=3,  # Change to 50 for full training
    validation_data=val_gen,
    validation_steps=50,
    callbacks=callbacks,
    verbose=1
)
